In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import matplotlib.pylab as pl

%matplotlib inline

### Intoduction

The following analysis used a dataset containing 1000 muon+ and 1000 electrons created at 1 GeV. The detector consisted of 2 thin silicon tracking detectors used to determine the charge of the particles, 5 liquid Argon calorimeters (as models for an electromagnetic calorimeter) and a block of iron serving as a muon detector.

The main assumption underlying the tracking procedure was that the particles were energetic enough to travel in a streight line. A 0.1 T magnetic field in the x direction was applied to produce tracking curvatures.

In [ ]:
#Electron data
eEnergyData = pd.read_csv( "electronData/output_nt_Energy.csv", comment="#",
                          names=[ "Truth", "Particle", "Track1", "Track2", "Layer1", "Layer2",
                                "Layer3", "Layer4", "Layer5", "Muon"], index_col = False )
ex1Data = pd.read_csv( "electronData/output_nt_Tracker1_x.csv", comment="#",
                          names=[ "EventID","x"])
ex2Data = pd.read_csv( "electronData/output_nt_Tracker2_x.csv", comment="#",
                           names=[ "EventID","x"])
ey1Data = pd.read_csv( "electronData/output_nt_Tracker1_y.csv", comment="#",
                           names=[ "EventID","y"])
ey2Data = pd.read_csv( "electronData/output_nt_Tracker2_y.csv", comment="#",
                           names=[ "EventID","y"])

#Proton data
pEnergyData = pd.read_csv( "protonData/output_nt_Energy.csv", comment="#",
                          names=[ "Truth", "Particle", "Track1", "Track2", "Layer1", "Layer2",
                                "Layer3", "Layer4", "Layer5", "Muon"], index_col = False )
px1Data = pd.read_csv( "protonData/output_nt_Tracker1_x.csv", comment="#",
                          names=[ "EventID","x"])
px2Data = pd.read_csv( "protonData/output_nt_Tracker2_x.csv", comment="#",
                           names=[ "EventID","x"])
py1Data = pd.read_csv( "protonData/output_nt_Tracker1_y.csv", comment="#",
                           names=[ "EventID","y"])
py2Data = pd.read_csv( "protonData/output_nt_Tracker2_y.csv", comment="#",
                           names=[ "EventID","y"])

#Muon+ data
mPlusEnergyData = pd.read_csv( "mu+Data/output_nt_Energy.csv", comment="#",
                          names=[ "Truth", "Particle", "Track1", "Track2", "Layer1", "Layer2",
                                "Layer3", "Layer4", "Layer5", "Muon"], index_col = False )
mPlusx1Data = pd.read_csv( "mu+Data/output_nt_Tracker1_x.csv", comment="#",
                          names=[ "EventID","x"])
mPlusx2Data = pd.read_csv( "mu+Data/output_nt_Tracker2_x.csv", comment="#",
                           names=[ "EventID","x"])
mPlusy1Data = pd.read_csv( "mu+Data/output_nt_Tracker1_y.csv", comment="#",
                           names=[ "EventID","y"])
mPlusy2Data = pd.read_csv( "mu+Data/output_nt_Tracker2_y.csv", comment="#",
                           names=[ "EventID","y"])

#Muon- data
mMinEnergyData = pd.read_csv( "mu-Data/output_nt_Energy.csv", comment="#",
                          names=[ "Truth", "Particle", "Track1", "Track2", "Layer1", "Layer2",
                                "Layer3", "Layer4", "Layer5", "Muon"], index_col = False )
mMinx1Data = pd.read_csv( "mu-Data/output_nt_Tracker1_x.csv", comment="#",
                          names=[ "EventID","x"])
mMinx2Data = pd.read_csv( "mu-Data/output_nt_Tracker2_x.csv", comment="#",
                           names=[ "EventID","x"])
mMiny1Data = pd.read_csv( "mu-Data/output_nt_Tracker1_y.csv", comment="#",
                           names=[ "EventID","y"])
mMiny2Data = pd.read_csv( "mu-Data/output_nt_Tracker2_y.csv", comment="#",
                           names=[ "EventID","y"])

In [ ]:
eEnergy = np.asarray(eEnergyData )
ex1 = np.asarray(ex1Data)
ey1 = np.asarray(ey1Data)
ex2 = np.asarray(ex2Data)
ey2 = np.asarray(ey2Data)

mPlusEnergy = np.asarray(mPlusEnergyData )
mPlusx1 = np.asarray(mPlusx1Data)
mPlusy1 = np.asarray(mPlusy1Data)
mPlusx2 = np.asarray(mPlusx2Data)
mPlusy2 = np.asarray(mPlusy2Data)

mMinEnergy = np.asarray(mMinEnergyData )
mMinx1 = np.asarray(mMinx1Data)
mMiny1 = np.asarray(mMiny1Data)
mMinx2 = np.asarray(mMinx2Data)
mMiny2 = np.asarray(mMiny2Data)

pEnergy = np.asarray(pEnergyData )
px1 = np.asarray(px1Data)
py1 = np.asarray(py1Data)
px2 = np.asarray(px2Data)
py2 = np.asarray(py2Data)

### Reconstruction

In [ ]:
#Get electron pairs.
electronPairs = []

for i in range (len(ex1)):
    for j in range (len(ex2)):
        if ex1[i,0] == ex2[j,0]:
            if (math.fabs(ex1[i,1] - ex2[j,1]) < 0.5):
                #Store as (x1, x2, y1, y2, EventID)
                electronPairs.append([ex1[i,1], ex2[j,1], ey1[i,1], ey2[j,1], ex1[i,0]]) 
                
#Get muon+ pairs.
muonPlusPairs = []

for i in range (len(mPlusx1)):
    for j in range (len(mPlusx2)):
        if mPlusx1[i,0] == mPlusx2[j,0]:
            if (math.fabs(mPlusx1[i,1] - mPlusx2[j,1]) < 2.5):
                #Store as (x1, x2, y1, y2, EventID)
                muonPlusPairs.append([mPlusx1[i,1], mPlusx2[j,1], mPlusy1[i,1], mPlusy2[j,1], mPlusx1[i,0]]) 
                
#Check using tracking.
electrons = []
muonPlus = []

particlePairs = np.concatenate((electronPairs, muonPlusPairs))

#Classify using tracking.
for pair in particlePairs:
    if pair[2] > pair[3]:
        electrons.append(pair)
    else:
        muonPlus.append(pair)
        
#Match with corresponding energy signature.
mPlusPairEnergy = []
for i in range (len(mPlusEnergy)):
    for pair in muonPlus:
        if pair[4] == i:
            mPlusPairEnergy.append(np.concatenate((mPlusEnergy[i], pair)))
            
ePairEnergy = []
for i in range (len(eEnergy)):
    for pair in electrons:
        if pair[4] == i:
            ePairEnergy.append(np.concatenate((eEnergy[i], pair)))

In [ ]:
eFrame = pd.DataFrame(data = ePairEnergy, index = None, columns = [ "Truth", "Particle", "Track1", 
                                                                  "Track2", "Layer1", "Layer2", 
                                                                  "Layer3", "Layer4", "Layer5", 
                                                                  "Muon", "x1", "x2", "y1", "y2", "ID"])

muPlusFrame = pd.DataFrame(data = mPlusPairEnergy, index = None, columns = [ "Truth", "Particle", "Track1", 
                                                                  "Track2", "Layer1", "Layer2", 
                                                                  "Layer3", "Layer4", "Layer5", 
                                                                  "Muon", "x1", "x2", "y1", "y2", "ID"])

### Validation

In [ ]:
from sklearn.preprocessing import StandardScaler
from tensorflow.python.keras.regularizers import l1_l2
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras.utils import plot_model
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Input, Dense, BatchNormalization, Activation, Dropout
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
from scikitplot.metrics import plot_roc
from bayes_opt import BayesianOptimization
import tensorflow as tf

In [ ]:
#Calibration Plots
eEnergySum = np.asarray([sum(Evalue[2:]) for Evalue in eEnergy])
mPlusEnergySum = np.asarray([sum(Evalue[2:]) for Evalue in mPlusEnergy])

eEnergyTruth = np.asarray(eEnergyData["Truth"])
mPlusEnergyTruth = np.asarray(mPlusEnergyData["Truth"])

eCalibMean = np.mean(eEnergyTruth/eEnergySum)
eCalib = eCalibMean*eEnergySum

mPlusCalibMean = np.mean(mPlusEnergyTruth/mPlusEnergySum)
mPlusCalib = mPlusCalibMean*mPlusEnergySum

eRatio = (eCalib - eEnergyTruth )/eEnergyTruth 
mPlusRatio = (mPlusCalib - mPlusEnergyTruth )/mPlusEnergyTruth 

n_bins = 20

fig, axs = plt.subplots(1, 2, sharey=True)

axs[0].hist(eRatio, bins=n_bins, range = (-0.2,0.2))
axs[0].set_xlabel('Calibrated')
axs[0].set_title('Electron')
axs[1].hist(mPlusRatio, bins=n_bins, range = (-0.2,0.2))
axs[1].set_xlabel('Calibrated')
axs[1].set_title('Muon+');

In [ ]:
print("Resolution:\nElectron = {:.4f}\nMuon+ = {:.4f}".format(np.std(eRatio), np.std(mPlusRatio)))

### Clasification model

In [ ]:
eTrain = np.asarray(eFrame[["Layer1", "Layer2","Layer3", "Layer4", "Layer5", "Muon", "y1", "y2"]])
eTarget = np.asarray(eFrame["Particle"]) - 1
muPlusTrain = np.asarray(muPlusFrame[["Layer1", "Layer2","Layer3", "Layer4", "Layer5", "Muon", "y1", "y2"]])
muPlusTarget = np.asarray(muPlusFrame["Particle"]) - 1

target = np.concatenate((eTarget[:1300], muPlusTarget[:1300]))
train = np.concatenate((eTrain[:1300], muPlusTrain[:1300]))

#Shuffle datasets
perm_train = np.random.permutation(train.shape[0])

train = train[perm_train]
target = target[perm_train]

#Standardise data.
train = StandardScaler().fit_transform(train)

#Split into train/test sets.
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size = 0.3)


In [ ]:
#Define testing model.

def get_model(nb_layer1, nb_layer2, nb_layer3, dropout1, dropout2):
    
    opts_dense = dict(activation = 'relu')
    
    i = Input(shape = 8, name = 'Input')
    x = Dense(nb_layer1, **opts_dense)(i)
    x = Dropout(dropout1)(x)
    x = Dense(nb_layer2, **opts_dense)(x)
    x = Dropout(dropout2)(x)
    x = Dense(nb_layer3, **opts_dense)(x)
    o = Dense(2, activation = 'softmax', name = 'Output')(x)
    
    return Model(i, o)

def fit_with(nb_layer1, nb_layer2, nb_layer3, dropout1, dropout2, lr):
     
    model = get_model(nb_layer1, nb_layer2, nb_layer3, dropout1, dropout2)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
    
    model.compile(optimizer = optimizer, 
            metrics = ['accuracy'], 
            loss = 'sparse_categorical_crossentropy')
    
    model.fit(x_train, y_train, epochs = 5, 
              verbose = 0, batch_size = 64, validation_data = [x_test, y_test], shuffle = True);
    
    loss, acc = model.evaluate(x_test, y_test)
    print('Loss: {:.4f}, Accuracy: {:.4f}'.format(loss, acc*100))
    
    return acc


#Peform optimisation.

pbounds = {'nb_layer1': (10,100), 'nb_layer2': (10,100), 'nb_layer3': (10,100), 
           'dropout1': (0.1,0.5), 'dropout2': (0.1,0.5), 'lr': (1e-4, 1e-2)}

optimizer = BayesianOptimization(f = fit_with, pbounds = pbounds, verbose = 2,  random_state = 1)

optimizer.maximize(init_points= 3, n_iter= 2)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)


In [ ]:
def DNN_Classifier(nb_feats, nb_target):
    
    opts_dense = dict(activation = 'relu')
    
    i = Input(shape = nb_feats, name = 'Input')
    x = Dense(59, **opts_dense)(i)
    x = Dropout(0.18)(x)               
    x = Dense(48, **opts_dense)(x)
    x = Dropout(0.24)(x)
    x = Dense(71, **opts_dense)(x)
    o = Dense(nb_target, activation = 'softmax', name = 'Output')(x)
    
    return Model(i, o, name = 'DNN_Classifier')

#Construct and compile model.

dnn = DNN_Classifier(x_train.shape[1], 3)
dnn.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.004), 
            metrics = ['accuracy'], 
            loss = 'sparse_categorical_crossentropy')

#Train model.
dnn.fit(x_train, y_train, epochs = 50, verbose = 1,
        batch_size = 64, validation_data = [x_test, y_test], shuffle = True);

In [ ]:
#Plot loss and save model.
dnn.save('dnn.h5')

loss = dnn.history.history['loss']
val_loss = dnn.history.history['val_loss']
epochs = list(range(len(dnn.history.history['loss'])))

plt.plot(epochs, loss, label = 'Loss')
plt.plot(epochs, val_loss, label = 'Val loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
#Evaluate model accuracy.
loss, acc = dnn.evaluate(x_test, y_test)
print('Model accuracy = {:.2f}%'.format(acc*100))

### Test over energy range

In [ ]:
eEnergyRange = pd.read_csv( "electron(500-10500)/output_nt_Energy.csv", comment="#",
                          names=[ "Truth", "Particle", "Track1", "Track2", "Layer1", "Layer2",
                                "Layer3", "Layer4", "Layer5", "Muon"], index_col = False )

eEnergyArray = np.asarray(eEnergyRange)

In [ ]:
eEnergySum = np.asarray([sum(Evalue[2:]) for Evalue in eEnergyArray])
eEnergyTruth = np.asarray(eEnergyRange["Truth"])

eCalibMean = np.mean(eEnergyTruth/eEnergySum)
eCalib = eCalibMean*eEnergySum
eRatio = (eCalib - eEnergyTruth )/eEnergyTruth 

pl.hist2d(eEnergyTruth, eRatio, bins = (10, 20))
pl.xlabel('True Energy [MeV]')
pl.ylabel('(Calibrated E − True E) / True E')
pl.show();

### Improvements discussion:

Preliminary runs failed to detect the muons with only 56% of the total energy deposited. An iron muon detector was added into the simulation in order to examine the muons more accuratelly. This allowed for a clear detection of muons up to 10GeV after which the particle passed through the detector. 

The machine learning model used in classification identified the 2 particle types (electrons and muons+) with 99.87% accuracy. This is unsurprising since the particles had opposite charges and therefore demonstrated curvature in opposite directions (determined by the y-coordinate). Additionally, muons deposited a significantly larger fraction of their energy in the iron muon detector compared to the electron which showered in the liquid Argon calorimeters depositing its entire energy. Therefore, the likely high lever features used to classify the particles were their spacial coordinates (specifically the y-coordinate) and energy signatures.

The thin silicon tracking detectors spacing was varied to allow for a clear curviture even at higher energies.
Various detector stuctures were tested (included in the Failed folder), however the one used to gather the data was chosen for its versitility to detect a range of particle types including muon+, muon-, electron and proton.